In [1]:
import yfinance as yf
import pandas as pd
from ipywidgets import Output
from sklearn.metrics import precision_score, accuracy_score
from sklearn.metrics import precision_score, accuracy_score
predictors = []

In [2]:
stocks = ["^gspc"]
data =pd.DataFrame([])
pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
first = True
               
for stock in stocks:
    stk = yf.Ticker(stock)  
    stock_data = stk.history(period='max') 
    stock_data=stock_data.loc["1990-1-1":].copy()
    del stock_data["Dividends"]
    del stock_data["Stock Splits"] 
    del stock_data["Open"]
    del stock_data["High"]
    del stock_data["Low"]
    stock_data["Tomarrow"] = stock_data["Close"].shift(-1)
    stock_data["Target"] = (stock_data["Close"]<stock_data["Tomarrow"]).astype(int)
    
    horizons = [2,5,60,250,1000]
    # predictors = []

    for horizon in horizons:
        rolling_averages = stock_data.rolling(horizon).mean()

        ratio_column = f"{stock}_Close_Avg_{horizon}"
        stock_data[ratio_column] = stock_data["Close"] / rolling_averages["Close"]

        trend_column = f"{stock}_Trend_{horizon}"
        stock_data[trend_column] = stock_data.shift(1).rolling(horizon).sum()["Target"]
        
        v_ratio_col =f"{stock}_Vol_Avg_{horizon}"
        stock_data[v_ratio_col] = stock_data["Volume"]/rolling_averages["Volume"]
        
        predictors+= [ratio_column, trend_column]
    
    if  first:
        first = False
    else:
        print("delete pred name")
        del stock_data['Close']
        del stock_data['Volume']
        del stock_data['Target']
    data = pd.concat([data,stock_data],ignore_index = False,axis=1)
    

In [3]:
data=data.dropna()

In [4]:
import sklearn
from sklearn.ensemble import RandomForestClassifier

train = data.iloc[:-100]
test = data.iloc[-100:]

In [5]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [6]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [7]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [8]:
predictions = backtest(data, model, predictors)

In [9]:
predictions["Predictions"].value_counts()

Predictions
0.00    4281
1.00     830
Name: count, dtype: int64

In [10]:
predictions["Target"].value_counts() / predictions.shape[0]

Target
1   0.54
0   0.46
Name: count, dtype: float64

In [11]:
prec = precision_score(predictions["Target"], predictions["Predictions"])
acc = accuracy_score(predictions["Target"], predictions["Predictions"])

In [12]:
print('precision: ',prec)
print('accuracy: ',acc)

precision:  0.5734939759036145
accuracy:  0.4791625904910976


In [13]:
feature_imp=pd.Series( model.feature_importances_).sort_index(ascending=True)

In [14]:
pred=pd.Series(predictors)

In [15]:
latest_pred = data.tail(1)

In [16]:
del latest_pred['Close']
del latest_pred['Volume']
del latest_pred['Tomarrow']
del latest_pred['Target']

In [18]:
latest_pred

,^gspc_Close_Avg_2,^gspc_Trend_2,^gspc_Vol_Avg_2,^gspc_Close_Avg_5,^gspc_Trend_5,^gspc_Vol_Avg_5,^gspc_Close_Avg_60,^gspc_Trend_60,^gspc_Vol_Avg_60,^gspc_Close_Avg_250,^gspc_Trend_250,^gspc_Vol_Avg_250,^gspc_Close_Avg_1000,^gspc_Trend_1000,^gspc_Vol_Avg_1000
Date,,,,,,,,,,,,,,,
2024-03-07 00:00:00-05:00,1.01,2.00,0.95,1.01,3.00,0.91,1.06,38.00,1.02,1.16,141.00,1.03,1.27,534.00,0.93


In [17]:
todays_pred = model.predict(latest_pred)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ^gspc_Vol_Avg_1000
- ^gspc_Vol_Avg_2
- ^gspc_Vol_Avg_250
- ^gspc_Vol_Avg_5
- ^gspc_Vol_Avg_60


In [ ]:
print(pd.concat([pred,feature_imp],axis=1))